# geoDB Access

The geoDB is a service provided by the [Euro Data Cube project](https://eurodatacube.com) (EDC) as a payed service. It comes
with a Python client that provides hugh level acess to your data and a certain amount of space in a PostGreSQL database. For managing (as sharing is) 
your data you will need a management (read/write) account to your database which you can purchase at the [EDC market place](https://hub.eox.at).

You can access the service in two ways:

- By using the Jupyter Python notebook provided by EOX (configuartion free, ```geodb = GeoDBClient()```)
- By using you own Jupyter notebook or Python script by providing a client id and secret to the GeoDBClient (```geodb = GeoDBClient(client_id="myid", client_secret="mysecet")```)

The client ID and secret is also provided by EOX in the latter case. You will find them in your EOX hub account section. You can also provide the credentials
via system environment varibles (GEODB_AUTH_CLIENT_ID and GEODB_AUTH_CLIENT_SECRET). These variables can be supplied via a .env file. 

There are two different types of geoDB accounts: a read only and a management (read/write) access. The system will determine your access right through your 
authentication credentials.


## Sharing Data

In [1]:
from xcube_geodb.core.geodb import GeoDBClient

##  Login from any maschine
### uncomment if not on EDC

Install xcube geoDB with command:
  
 `conda install xcube_geodb -c conda-forge`


In [3]:
#client_id=YourID
#client_secret=YourSecret
#geodb = GeoDBClient(client_id=client_id, client_secret=client_secret, auth_mode="client-credentials")

## Login in EDC environment
### comment if not on EDC

In [5]:
#geodb = GeoDBClient()

### Get your user name

In [6]:
geodb.whoami

'geodb_ci_test_user'

In [7]:
geodb._auth_aud

'https://geodb.brockmann-consult.de'

In [8]:
import geopandas

# Have a look at fiona feature schema
collections = {
        "land_use":
        {
            "crs": 3794,
            "properties":
            {
                "RABA_PID": "float",
                "RABA_ID": "float",
                "D_OD": "date"
            }
        }
    }


geodb.create_collections(collections, clear=True)

gdf = geopandas.read_file('data/sample/land_use.shp')
geodb.insert_into_collection('land_use', gdf.iloc[:100,:]) # minimizing rows to 100, if you are in EDC, you dont need to make the subset.

Processing rows from 0 to 100


100 rows inserted into land_use

### Publish a Collection to the World

In [9]:
geodb.list_my_grants()

,Grants
0,No Grants


Please change the second positional kwargs to the geodb user you want to grant access to, if you are on EDC, go ahead and use 'geodb_test5' user:

In [10]:
geodb.grant_access_to_collection("land_use", "geodb_admin")

Access granted on land_use to geodb_admin

In [11]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_ci_test_user,land_use,geodb_admin,SELECT


### Accessing the Collection as a different User

Let's access the collection as a different user (a test user in this case). You should now see a land_use collection.

In [12]:
geodb = GeoDBClient(client_id="93wdyTwOJebDozSCB4CubsWdFVKrSaXY", client_secret="zv0h3D3s90KAL640bw_MU4XMb6DfpJQloh877mH6Z1Nu4NrgPU2wSG3Ur_poUtqy", auth_mode="client-credentials")
geodb.whoami

'geodb_test5'

In [13]:
geodb.get_my_collections()

,owner,database,table_name
0,anja,anja,E1
1,anja,anja,E10a1
2,anja,anja,E10a2
3,anja,anja,E11
4,anja,anja,E1a
...,...,...,...
61,geodb_admin,public,land_use
62,geodb_ciuser,geodb_ciuser,land_use
63,geodb_d2c4722a-cc19-4ec1-b575-0cdb6876d4a7,lpis_iacs,land_use_slo
64,geodb_d2c4722a-cc19-4ec1-b575-0cdb6876d4a7,lpis_iacs,lpis_aut


### Revoke access

Let's go back to the original user.

In [16]:
geodb = GeoDBClient()
geodb.whoami


'geodb_ci_test_user'

In [17]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_ci_test_user,land_use,geodb_admin,SELECT


In [18]:
geodb.revoke_access_from_collection("land_use", 'geodb_admin')

Access revoked from geodb_ci_test_user on land_use

In [19]:
geodb.list_my_grants()

,Grants
0,No Grants


### Accessing the Collection as a different User (on EDC)

Let's list my collections again as a different user (a test user in this case). You should not see the above land use collection.

In [ ]:
#geodb = GeoDBClient(client_id="93wdyTwOJebDozSCB4CubsWdFVKrSaXY", client_secret="zv0h3D3s90KAL640bw_MU4XMb6DfpJQloh877mH6Z1Nu4NrgPU2wSG3Ur_poUtqy", auth_mode="client-credentials")
#geodb.whoami

In [ ]:
#geodb.get_my_collections()

### Finally going back to the initial user and deleting the collection:


In [21]:
geodb.drop_collection('land_use')

Collection ['geodb_ci_test_user_land_use'] deleted